In [ ]:
import flwr as fl
import torch
import torch.optim as optim
import pandas as pd
import numpy as np
from agent import PolicyNetwork
from env import DroneEnv

In [9]:
csv_path = 'data/sensor_dataset.csv'

In [ ]:

df = pd.read_csv(csv_path, sep=';')

trajectories = []
for uid, group in df.groupby('uid'):
    group = group.sort_values('timestamp').reset_index(drop=True)
    for i in range(len(group) - 1):
        curr = group.iloc[i]
        next_ = group.iloc[i + 1]

        state = np.array([
            curr['position_x'],
            curr['position_y'],
            0, # coverage
            curr['battery_voltage'] / 20.0,  # normalização
            curr['wind_speed'] / 20.0,
            curr['wind_angle'] / 360.0,
            curr['altitude'] / 1000.0,
            curr['yaw'] / 360.0,
        ], dtype=np.float32)

        # Próximo estado
        next_state = np.array([
            next_['position_x'] / 1000.0,
            next_['position_y'] / 1000.0,
            0.0,
            next_['battery_voltage'] / 20.0,
            next_['wind_speed'] / 20.0,
            next_['wind_angle'] / 360.0,
            next_['altitude'] / 1000.0,
            next_['yaw'] / 360.0,
        ], dtype=np.float32)

        # Ação aproximada = delta posição
        dx = (next_['position_x'] - curr['position_x']) / 10.0
        dy = (next_['position_y'] - curr['position_y']) / 10.0
        action = np.array([dx, dy], dtype=np.float32)

        # Recompensa estimada baseada nos critérios usados no ambiente
        reward = 0
        if i > 0:
            redundancy = 1 if np.linalg.norm([dx, dy]) < 1e-2 else 0
            reward += 1 - redundancy
            reward -= 0.5 * (curr['wind_speed'] / 20.0)
            if curr['battery_voltage'] < 15:
                reward -= 0.3

        log_prob = 0  # Placeholder: pode ser ignorado no fine-tuning
        done = False  # Assumimos episódio contínuo para fine-tuning

        trajectories.append((state, action, reward, done, log_prob))

In [5]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
STATE_DIM = 7
ACTION_DIM = 2

In [6]:

class DroneClient(fl.client.NumPyClient):
    def __init__(self):
        self.model = PolicyNetwork(STATE_DIM, ACTION_DIM).to(DEVICE)

    def get_parameters(self, config):
        return [val.cpu().numpy() for val in self.model.state_dict().values()]

    def set_parameters(self, parameters):
        keys = list(self.model.state_dict().keys())
        state_dict = {k: torch.tensor(v) for k, v in zip(keys, parameters)}
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)

        env = DroneEnv()
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)

        for episode in range(5):  # Local episodes
            state = torch.tensor(env.reset(), dtype=torch.float32).to(DEVICE)
            done = False
            while not done:
                action, log_prob = self.model.act(state)
                next_state, reward, done, _ = env.step(action.cpu().detach().numpy())
                next_state = torch.tensor(next_state, dtype=torch.float32).to(DEVICE)

                loss = -log_prob * reward
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                state = next_state

        return self.get_parameters({}), 1, {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        env = DroneEnv()

        total_reward = 0.0
        for _ in range(5):
            state = torch.tensor(env.reset(), dtype=torch.float32).to(DEVICE)
            done = False
            while not done:
                with torch.no_grad():
                    action, _ = self.model.act(state)
                next_state, reward, done, _ = env.step(action.cpu().numpy())
                total_reward += reward
                state = torch.tensor(next_state, dtype=torch.float32).to(DEVICE)

        avg_reward = total_reward / 5
        return -avg_reward, 1, {"reward": avg_reward}

In [7]:
fl.client.start_numpy_client(server_address="localhost:8080", client=DroneClient())


	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8080: Failed to connect to remote host: connect: Connection refused (111)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8080: Failed to connect to remote host: connect: Connection refused (111)", grpc_status:14}"
>